In [1]:
# ── 필수 모듈 ────────────────────────────────────────────────
from pathlib import Path
import json
import pandas as pd
from darts import TimeSeries
from tqdm import tqdm       # 진행률 표시(선택)

# ── 폴더 위치 지정 ───────────────────────────────────────────
PROJECT_ROOT = Path("matrix-nilm-test") / "matrix-nilm-test"   # 루트 폴더
CSV_ROOT  = PROJECT_ROOT / "csv"
JSON_ROOT = PROJECT_ROOT / "json"
SAVE_DIR  = PROJECT_ROOT / "ts_pickle"
SAVE_DIR.mkdir(parents=True, exist_ok=True)


In [2]:

# ── 보조 함수 : active_inactive 마스크 ───────────────────────
def build_active_mask(times: pd.Series, intervals: list[list[str]]) -> pd.Series:
    mask = pd.Series(0, index=times.index, dtype="int8")
    for start, end in intervals:
        s, e = pd.to_datetime(start), pd.to_datetime(end)
        mask |= ((times >= s) & (times <= e)).astype("int8")
    return mask

# ── CSV 1개 처리 함수 ───────────────────────────────────────
def process_one(csv_path: Path) -> None:
    out_path = SAVE_DIR / f"{csv_path.stem}.ts.pkl"
    if out_path.exists():          # 이미 처리됨 → 건너뛰기
        return

    # 대응 JSON: CSV_ROOT 이후 경로 + .json, json 루트로 교체
    rel   = csv_path.relative_to(CSV_ROOT)
    json_path = (JSON_ROOT / rel).with_suffix(".json")
    if not json_path.exists():
        print(f"[SKIP] {csv_path} → JSON 없음")
        return                    # JSON 없으면 전체 건너뛰기

    # ── 1. CSV 읽기 & 5개 컬럼만 ──────────────────────────
    cols = ["date_time", "active_power", "voltage", "current", "reactive_power"]
    df = pd.read_csv(csv_path, usecols=cols, parse_dates=["date_time"])

    # ── 2. 1 ms 샘플링(행 % 3 == 0) ──────────────────────
    df = df.iloc[::3].reset_index(drop=True)

    # ── 3. JSON 라벨 로드 & active_inactive 생성 ───────────
    with open(json_path, encoding="utf-8") as f:
        label = json.load(f)

    # JSON 구조가 다른 경우를 대비해 두 가지 키 모두 시도
    intervals = (
        label.get("labels", {}).get("active_inactive")
        or label.get("active_inactive")
        or []
    )
    if not intervals:
        print(f"[WARN] {json_path} : active_inactive 라벨 없음 → 전부 0")
    df["active_inactive"] = build_active_mask(df["date_time"], intervals)

    # ── 4. Darts TimeSeries 변환 & 5. 피클 저장 ───────────
    ts = TimeSeries.from_dataframe(
        df,
        time_col="date_time",
        value_cols=[
            "active_power", "voltage", "current",
            "reactive_power", "active_inactive"
        ],
        fill_missing_dates=False
    )
    ts.to_pickle(out_path)

# ── 전체 일괄 실행 ───────────────────────────────────────────
for csv in tqdm(sorted(CSV_ROOT.rglob("*.csv"))):
    process_one(csv)


  0%|          | 0/167 [00:00<?, ?it/s]

[SKIP] matrix-nilm-test\matrix-nilm-test\csv\house_001\H001_ch01_20230924.csv → JSON 없음


  1%|          | 2/167 [00:02<03:49,  1.39s/it]

[SKIP] matrix-nilm-test\matrix-nilm-test\csv\house_001\H001_ch02_20230924.csv → JSON 없음


  2%|▏         | 4/167 [00:05<03:38,  1.34s/it]

...

[WARN] matrix-nilm-test\matrix-nilm-test\json\house_007\H007_ch23_20231002.json : active_inactive 라벨 없음 → 전부 0


 77%|███████▋  | 128/167 [04:49<01:46,  2.73s/it]

[WARN] matrix-nilm-test\matrix-nilm-test\json\house_008\H008_ch03_20231002.json : active_inactive 라벨 없음 → 전부 0


 77%|███████▋  | 128/167 [04:50<01:28,  2.27s/it]


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [4]:
# -*- coding: utf-8 -*-
"""
CSV  ➜  라벨링·다운샘플  ➜  Darts TimeSeries  ➜  pickle
"""

import json, pathlib, glob, os
import pandas as pd
from tqdm import tqdm
from darts import TimeSeries

# ─────────────────── 폴더 경로 ─────────────────────────────
ROOT      = pathlib.Path(r"matrix-nilm-test/matrix-nilm-test")
CSV_ROOT  = ROOT / "csv"          # csv/house_001/…
JSON_ROOT = ROOT / "json"         # json/house_001/…
SAVE_DIR  = ROOT / "ts_pickle"    # 결과 저장
SAVE_DIR.mkdir(parents=True, exist_ok=True)

# ─────────────────── 1. JSON → 활성 구간 ───────────────────
def read_active_windows(json_path: pathlib.Path):
    """사용자 기존 함수 그대로"""
    with open(json_path, encoding="utf-8") as f:
        data = json.load(f)
    return [
        (pd.to_datetime(s), pd.to_datetime(e))
        for s, e in data["labels"]["active_inactive"]
    ]

# ─────────────────── 2. CSV 1개 처리 ───────────────────────
def process_one(csv_path: pathlib.Path):
    out_path = SAVE_DIR / f"{csv_path.stem}.ts.pkl"
    if out_path.exists():                  # 이미 처리된 파일은 패스
        return

    # house·채널·날짜 파싱
    house_id, ch_id, date_str, *_ = csv_path.stem.split("_")   # H001 ch04 20230924
    # ① 날짜가 같은 JSON 우선, ② 없으면 같은 house+채널 JSON 아무거나
    json_path = (
        JSON_ROOT / f"house_{house_id[1:]}" /
        f"{house_id}_{ch_id}_{date_str}.json"
    )
    if not json_path.exists():
        # 날짜 다른 JSON 검색
        pattern = (
            JSON_ROOT / f"house_{house_id[1:]}" /
            f"{house_id}_{ch_id}_*.json"
        )
        hits = list(glob.glob(str(pattern)))
        if hits:
            json_path = pathlib.Path(hits[0])
        else:
            print(f"[SKIP] {csv_path}  ▶ JSON 없음")
            return

    # ── CSV 로드 & 5 컬럼 ↓ ──────────────────────────────
    keep = ["date_time", "active_power", "voltage", "current", "reactive_power"]
    df = pd.read_csv(csv_path, usecols=keep, parse_dates=["date_time"])

    # ── 1 ms 등간격 샘플링(행 % 3 == 0) ─────────────────
    df = df.iloc[::3].reset_index(drop=True)

    # ── active_inactive 라벨링 ──────────────────────────
    windows = read_active_windows(json_path)
    mask = pd.Series(0, index=df.index, dtype="int8")
    for s, e in windows:                         # start ≤ t ≤ end
        mask |= ((df["date_time"] >= s) & (df["date_time"] <= e)).astype("int8")
    df["active_inactive"] = mask

    # ── Darts TimeSeries 변환 & 저장 ───────────────────
    ts = TimeSeries.from_dataframe(
        df,
        time_col="date_time",
        value_cols=[
            "active_power", "voltage", "current",
            "reactive_power", "active_inactive"
        ],
        fill_missing_dates=False
    )
    ts.to_pickle(out_path)

# ─────────────────── 3. 전체 실행 ─────────────────────────
csv_list = sorted(CSV_ROOT.rglob("*.csv"))
print(f"총 {len(csv_list):,} 개 CSV 처리 시작")

for csv in tqdm(csv_list):
    process_one(csv)


총 167 개 CSV 처리 시작


100%|██████████| 167/167 [10:31<00:00,  3.78s/it]


In [21]:
from pathlib import Path
from darts import TimeSeries
import pandas as pd

def ts_pkl_to_df(pkl_path: str | Path) -> pd.DataFrame:
    """
    .ts.pkl → pandas.DataFrame 로 변환
    """
    pkl_path = Path(pkl_path)
    if not pkl_path.exists():
        raise FileNotFoundError(pkl_path)

    # ✅ TimeSeries 복구
    ts: TimeSeries = TimeSeries.from_pickle(str(pkl_path))   # ← 여기!

    # pandas DataFrame 으로
    return ts.to_dataframe()   # 또는 ts.to_dataframe()

# ── 사용 예시 ───────────────────────────────────────────────
if __name__ == "__main__":
    pkl_file = "matrix-nilm-test/matrix-nilm-test/ts_pickle/H001_ch01_20230924.ts.pkl"
    df = ts_pkl_to_df(pkl_file)
    display(df)


component,active_power,voltage,current,reactive_power,active_inactive
date_time,,,,,
2023-09-24 00:00:00.000,302.59,217.0,2.187,365.60,0.0
2023-09-24 00:00:00.100,301.21,217.0,2.180,364.77,0.0
2023-09-24 00:00:00.200,303.24,217.0,2.178,362.52,0.0
2023-09-24 00:00:00.300,303.94,217.0,2.187,364.48,0.0
2023-09-24 00:00:00.400,303.30,217.0,2.183,363.88,0.0
...,...,...,...,...,...
2023-09-24 23:59:59.500,264.41,218.0,1.834,299.89,0.0
2023-09-24 23:59:59.600,264.41,218.0,1.834,299.89,0.0
2023-09-24 23:59:59.700,264.41,218.0,1.834,299.89,0.0
